In [23]:
from langchain.document_loaders import PyPDFDirectoryLoader,UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_huggingface.llms.huggingface_endpoint import HuggingFaceEndpoint # Now the import should work
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModel,pipeline,AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

In [2]:
import torch

In [3]:
URLs=[
   'https://rosabon-finance.com/',
   'https://rosabon-finance.com/about/who-we-are',
   'https://rosabon-finance.com/investment/rosabon-treasury-note',
   'https://rosabon-finance.com/investment/rosabon-win-big-note'
]

loader = UnstructuredURLLoader(urls=URLs)
data = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [5]:
text_chunks = text_splitter.split_documents(data)

In [6]:
text_chunks

[Document(metadata={'source': 'https://rosabon-finance.com/'}, page_content="Secure a financially free future\n\nEarn as high as 19% interest on your placed funds\n\nStart Investing Now\n\nAcquire Your Dream Vehicle\n\nWith our affordable lease solutions\n\nGet A Lease Today\n\nAccess Loan Within 24 Hours\n\nAccess up to N6,000,000 within 24 hours!\n\nGet A Loan Today\n\nWe are now live nationwide!\n\nAccess up to N6,000,000 within 24 hours!\n\nGet A Loan Today\n\n\n\nCBN\n\nLicensed by the Central Bank of Nigeria\n\nNigeria's leading Non-Bank Financial Intermediary\n\n100k+\n\nActive Customers"),
 Document(metadata={'source': 'https://rosabon-finance.com/'}, page_content="N201+\n\nBillion Credit Processed\n\n\n\nDesign\n\n\n\nDesign\n\nGet Ahead with Lifestyle Solutions Tailored just for you\n\nExperience Unlimited Access to Financial Opportunities, Independence and Growth with Rosabon, Today.\n\n\n\n\n\nLoans\n\nNeed funds for life's opportunities? Get up to ₦100 Million with our has

In [15]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")


In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [17]:
llm.invoke("who is TInubu")

AIMessage(content='Bola Ahmed Tinubu is a Nigerian politician who served as the Governor of Lagos State from 1999 to 2007 and is the current president of Nigeria.  He assumed the presidency on May 29, 2023.  Before his governorship, he was a senator representing Lagos West Senatorial District during the brief Third Republic.  Tinubu is a prominent member of the All Progressives Congress (APC) party and played a significant role in its formation.  He is often referred to as a national leader within the party.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-2599a3d1-5845-4e69-9005-a9d1d52d38d0-0', usage_metadata={'input_tokens': 5, 'output_tokens': 113, 'total_tokens': 118, 'input_token_details': {'cache_read': 0}})

In [22]:



emb_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [26]:
vector_store = FAISS.from_documents(text_chunks, emb_model)
    
retriever = vector_store.as_retriever()

In [28]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001872C5011E0>, search_kwargs={})

In [38]:
query = "Tell me about RWIN"
docs=vector_store.similarity_search(query)


In [33]:
docs

[Document(id='913c8975-f37e-448c-9f85-30fd85b130d4', metadata={'source': 'https://rosabon-finance.com/investment/rosabon-treasury-note'}, page_content='Thanks to every good work man/woman of rosabon finance. I want to say that I feel good doing business with you. My appreciation to Peter Okonkwor, you are real and legit in your job, every customer would want to work with you. The business with rosabon finance which started on 30 of April 2020 has officially ended 30 of March 2021 in good relationship. - Rosemary\n\nJoin thousands of investors on our platform today and start growing your wealth with confidence.'),
 Document(id='5d3f16c7-a90c-44f1-9edb-e5e9409c6dd9', metadata={'source': 'https://rosabon-finance.com/investment/rosabon-win-big-note'}, page_content='Thanks to every good work man/woman of rosabon finance. I want to say that I feel good doing business with you. My appreciation to Peter Okonkwor, you are real and legit in your job, every customer would want to work with you. T

In [34]:
query_retriever_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [39]:
result = query_retriever_chain(query)

In [40]:
print(result['result'])

RWIN (Rosabon Win Big Note) is a 30-day investment product that offers a 2% interest rate on every N20,000 invested.  Investors are also entered into a monthly raffle draw with a grand prize of N2,500,000.


In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a helpful and informative AI assistant. "
    "Your task is to answer questions accurately and concisely using the provided context. "
    "If the context does not contain the answer, admit that you don't know. "
    "Avoid speculation or making up answers. "
    "Focus on extracting relevant information from the context and presenting it in a clear and understandable way. "
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [49]:
response = rag_chain.invoke({"input": "Tell me everything I Need to know about RWIN"})
print(response["answer"])

RWIN (Rosabon Win Big Note) is a 30-day investment with a 2% interest rate on every N20,000 invested.  Investors are also entered into a monthly raffle draw with a grand prize of N2,500,000.
